# SCOPのテキストをダウンロード

In [ ]:
import urllib.request
import os

scop = "http://scop.mrc-lmb.cam.ac.uk/files/scop-cla-latest.txt"
with urllib.request.urlopen(scop) as content:
    contents = content.read()
    html = contents.decode()

path = "./data/database"
with open(path, "w") as f:
    f.write(html)

# コンタクトマップ作成

In [ ]:
import urllib.request, urllib.error
import sys
import os
import numpy as np
from scipy.spatial import distance
import matplotlib.pyplot as plt
from tqdm import tqdm
scop = "./data/database"
with open(scop) as f:
    slst = f.read().splitlines()

for i in tqdm(slst):
    if i[0] == "#":
        continue
    line = list(i.split())
    name = line[1]   #nameはPDBのID
    area = line[2]
    if "A" in area[3:] or 'B' in area[3:] or 'S' in area[3:] or'I' in area[3:]or 'P'in area[3:]:
        continue #アルファベットが2桁の場合はPDBフォーマットが崩れる
    if "," in area:
        continue #領域が複数ある場合は排除
    chain_num = area.index(":")
    chain = area[:chain_num]   #chainを得る
    st_num = area[3:].index("-") #残基番号がマイナスの場合を考慮するため
    st_num += 3
    st = int(area[chain_num+1:st_num])
    end = int(area[st_num+1:])
    if end > 1000:
        continue #1000以上はPDBフォーマットが崩れる
    length = end - st + 1    #配列の長さ
    if length < 128:
        continue
    #pdbファイルを保存
    url = "https://files.rcsb.org/download/" + name + ".pdb"
    path_p = "./pdb/" + name + ".pdb" #pdbファイルの保存先
    if not (os.path.exists(path_p)):   
        try:
            with urllib.request.urlopen(url) as content:
                contents = content.read()
                html = contents.decode()
            with open(path_p, "w") as f:
                f.write(html)
        except:
            continue
    with open(path_p) as f:
            lst = f.read().splitlines()
    CA_list = [[0.0001] * 3 for _ in range(length)]
    for j in (lst):
        pdb_line = list(j.split())
        if len(pdb_line) == 12:
            if (pdb_line[0] == 'ATOM' or pdb_line[0] == 'HETATM') and pdb_line[4] == chain:
                try:
                    int(pdb_line[5])
                except:
                    continue
                if int(pdb_line[5]) < st:
                    continue #SCOPの範囲に満たないものを排除
                if int(pdb_line[5]) > end:
                    break #ミッシング残基があり、SCOPの範囲を越えてしまった場合
                if pdb_line[2] == 'CA':
                    CA_list[int(pdb_line[5])-st] = pdb_line[6:9]
                    if int(pdb_line[5])==end:
                        break
    protein = False #構造ドメインを距離行列にできるかのチェック
    for k in range(length//128):
        CA = []
        for l in range(128):
            CA.append(CA_list[l+k*128])
        if [0.0001]*3 in CA: #ミッシング残基がある場合は排除
            continue
        CA = np.array(CA)
        dist = distance.cdist(CA, CA, metric='euclidean')
        dist = np.array(dist)
        plt.imshow(-dist)
        np.save("./dataset/"+name+chain+str(st)+str(k), dist)